<a href="https://colab.research.google.com/github/Lawrence-Krukrubo/Building-a-Content-Based-Movie-Recommender-System/blob/master/building_a_content_based_recommendation_system.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 align="center"><font size="5">CONTENT-BASED FILTERING</font></h1>

Recommendation systems are a collection of algorithms used to recommend items to users based on information taken from the user. These systems have become ubiquitous, and can be commonly seen in online stores, movies databases and job finders. In this notebook, we will explore Content-based recommendation systems and implement a simple version of one using Python and the Pandas library.

In [1]:
# For creating and manipulating structured tabular data
import pandas as pd

Let's set maximum rows to be displayed at any time to not more than 20

In [2]:
pd.set_option('display.max_rows', 20)

### Saving the raw files from github

Both files have been saved in raw .csv format in  the code cell below, but if you want to download directly from the website, click this [link](https://grouplens.org/datasets/movielens/) and <br>
Select the file name 'ml-latest-small.zip (size: 1 MB)'

In [3]:
movies_data = 'https://raw.githubusercontent.com/Blackman9t/EDA/master/movies.csv'
ratings_data = 'https://raw.githubusercontent.com/Blackman9t/EDA/master/ratings.csv'

### Defining additional NaN values

In [4]:
missing_values = ['na','--','?','-','None','none','non']

### Reading the data to the data frame

In [5]:
movies_df = pd.read_csv(movies_data, na_values=missing_values)
ratings_df = pd.read_csv(ratings_data, na_values=missing_values)

HTTPError: HTTP Error 404: Not Found

In [11]:
print('Movies_df Shape:',movies_df.shape)
movies_df

NameError: name 'movies_df' is not defined

In [0]:
print('Ratings_df Shape:',ratings_df.shape)
ratings_df.head()

Ratings_df Shape: (100836, 4)


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


### Let's first explore and prepare the movies_df

Let's remove the year from the title column and place it in its own column, using the handy [extract](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.Series.str.extract.html) function of pandas, alongside python regex.

In [0]:
#Using regular expressions to find a year stored between parentheses
#We specify the parantheses so we don't conflict with movies that have years in their titles
movies_df['year'] = movies_df.title.str.extract('(\(\d\d\d\d\))',expand=False)
movies_df.head(3)

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,(1995)
1,2,Jumanji (1995),Adventure|Children|Fantasy,(1995)
2,3,Grumpier Old Men (1995),Comedy|Romance,(1995)


In [0]:
#Removing the parentheses
movies_df['year'] = movies_df.year.str.extract('(\d\d\d\d)',expand=False)
movies_df.head(3)

,movieId,title,genres,year
0,1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji (1995),Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men (1995),Comedy|Romance,1995


In [0]:
#Removing the years from the 'title' column
movies_df['title'] = movies_df.title.str.replace('(\(\d\d\d\d\))', '')
movies_df.head(3)

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995


In [0]:
#Applying the strip function to get rid of any ending whitespace characters that may have appeared
movies_df['title'] = movies_df['title'].apply(lambda x: x.strip())
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,Adventure|Animation|Children|Comedy|Fantasy,1995
1,2,Jumanji,Adventure|Children|Fantasy,1995
2,3,Grumpier Old Men,Comedy|Romance,1995
3,4,Waiting to Exhale,Comedy|Drama|Romance,1995
4,5,Father of the Bride Part II,Comedy,1995


With that, let's also split the values in the Genres column into a list of Genres to simplify future use. This can be achieved by applying Python's split string function on the correct column.

In [0]:
#Every genre is separated by a | so we simply have to call the split function on |
movies_df['genres'] = movies_df.genres.str.split('|')
movies_df.head()

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995
4,5,Father of the Bride Part II,[Comedy],1995


Let's view summary of the data, the memory consumption and if the titles are arranged logically

In [0]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9742 entries, 0 to 9741
Data columns (total 4 columns):
movieId    9742 non-null int64
title      9742 non-null object
genres     9742 non-null object
year       9729 non-null object
dtypes: int64(1), object(3)
memory usage: 304.5+ KB


In [0]:
movies_df_original_mem = movies_df.memory_usage()
movies_df_original_mem

Index         80
movieId    77936
title      77936
genres     77936
year       77936
dtype: int64

In [0]:
# Let's convert movieId column from int64 to int8 to save memory space
movies_df.movieId = movies_df.movieId.astype('int32')

Let's check for missing values

In [0]:
movies_df.isna().sum()

movieId     0
title       0
genres      0
year       13
dtype: int64

let's fill movies_df missing year  values with 0 to indicate the year is not readily available. we have only 13 rows 

In [0]:
movies_df.year.fillna(0, inplace=True)

In [0]:
# Let's now convert year column from int6a to int8, since it holds a max of just 4 digits of numbers. Thereby saving space.
movies_df.year = movies_df.year.astype('int16')

In [0]:
movies_df_new_mem = movies_df.memory_usage()

print(movies_df_original_mem)
print()
print(movies_df_new_mem)

Index         80
movieId    77936
title      77936
genres     77936
year       77936
dtype: int64

Index         80
movieId    38968
title      77936
genres     77936
year       19484
dtype: int64


Let's see a summary of the data types again

In [0]:
movies_df.dtypes

movieId     int32
title      object
genres     object
year        int16
dtype: object

In [0]:
movies_df.head(3)

,movieId,title,genres,year
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995
2,3,Grumpier Old Men,"[Comedy, Romance]",1995


Now, let's  One-Hot-Encode the list of genres. This encoding is needed for feeding categorical data. In this case, we store every different genre in columns that contain either 1 or 0. 1 shows that a movie has that genre and 0 shows that it doesn't. Let's also store this dataframe in another variable, just incase we need the one without genres at some point.


In [0]:
# First let's make a copy of the movies_df
movies_with_genres = movies_df.copy(deep=True)

# Let's iterate through movies_df, then append the movie genres as columns of 1s or 0s.
# 1 if that column contains movies in the genre at the present index and 0 if not.

x = []
for index, row in movies_df.iterrows():
    x.append(index)
    for genre in row['genres']:
        movies_with_genres.at[index, genre] = 1

# Confirm that every row has been iterated and acted upon
print(len(x) == len(movies_df))

movies_with_genres.head(3)

True


,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,NaN,NaN,NaN,1.0,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [0]:
#Filling in the NaN values with 0 to show that a movie doesn't have that column's genre
movies_with_genres = movies_with_genres.fillna(0)
movies_with_genres.head(3)

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Let's look at the ratings data set now

In [0]:
# print out the shape and first five rows of ratings data.
print('Ratings_df shape:',ratings_df.shape)          
ratings_df.head()

Ratings_df shape: (100836, 4)


,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [0]:
# Dropping the timestamp column
ratings_df.drop('timestamp', axis=1, inplace=True)

# Confirming the drop
ratings_df.head(3)

,userId,movieId,rating
0,1,1,4.0
1,1,3,4.0
2,1,6,4.0


In [0]:
# Let's confirm the right data types exist per column in ratings data_set

ratings_df.dtypes

userId       int64
movieId      int64
rating     float64
dtype: object

In [0]:
# Let's check for missing values

ratings_df.isna().sum()

userId     0
movieId    0
rating     0
dtype: int64

## Content Based recommender System

Now, let's implement a Content-Based or Item-Item recommendation systems. This technique attempts to figure out what a user's favourite aspects of an item is, and then recommends items that present those aspects. 

Let's begin by creating an input user to recommend movies to. The user's name will be Lawrence and we would assume Lawrence has rated the following movies with the following ratings:-

Notice: To add more movies, simply increase the amount of elements in the userInput. Feel free to add more in! Just be sure to write it in with capital letters and if a movie starts with a "The", like "The Matrix" then write it in like this: 'Matrix, The' .

Step 1: Creating Lawrence's Profile

In [0]:
# so on a scale of 0 to 5, with 0 min and 5 max, see Lawrence's movie ratings below
Lawrence_movie_ratings = [
            {'title':'Predator', 'rating':4.9},
            {'title':'Final Destination', 'rating':4.9},
            {'title':'Mission Impossible', 'rating':4},
            {'title':"Beverly Hills Cop", 'rating':3},
            {'title':'Exorcist, The', 'rating':4.8},
            {'title':'Waiting to Exhale', 'rating':3.9},
            {'title':'Avengers, The', 'rating':4.5},
            {'title':'Omen, The', 'rating':5.0}
         ] 
Lawrence_movie_ratings = pd.DataFrame(Lawrence_movie_ratings)
Lawrence_movie_ratings

,rating,title
0,4.9,Predator
1,4.9,Final Destination
2,4.0,Mission Impossible
3,3.0,Beverly Hills Cop
4,4.8,"Exorcist, The"
5,3.9,Waiting to Exhale
6,4.5,"Avengers, The"
7,5.0,"Omen, The"


Add movieId to input user
With the input complete, let's extract the input movie's ID's from the movies dataframe and add them into it.

We can achieve this by first filtering out the rows that contain the input movie's title and then merging this subset with the input dataframe. We also drop unnecessary columns for the input to save memory space.

In [0]:
# Extracting movie Ids from movies_df and updating lawrence_movie_ratings with movie Ids.

Lawrence_movie_Id = movies_df[movies_df['title'].isin(Lawrence_movie_ratings['title'])]

# Merging Lawrence movie Id and ratings into the lawrence_movie_ratings data frame. 
# This action implicitly merges both data frames by the title column.

Lawrence_movie_ratings = pd.merge(Lawrence_movie_Id, Lawrence_movie_ratings)

# Display the merged and updated data frame.

Lawrence_movie_ratings

,movieId,title,genres,year,rating
0,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,3.9
1,1350,"Omen, The","[Horror, Mystery, Thriller]",1976,5.0
2,45662,"Omen, The","[Horror, Thriller]",2006,5.0
3,1997,"Exorcist, The","[Horror, Mystery]",1973,4.8
4,2153,"Avengers, The","[Action, Adventure]",1998,4.5
5,89745,"Avengers, The","[Action, Adventure, Sci-Fi, IMAX]",2012,4.5
6,3409,Final Destination,"[Drama, Thriller]",2000,4.9
7,3527,Predator,"[Action, Sci-Fi, Thriller]",1987,4.9
8,4085,Beverly Hills Cop,"[Action, Comedy, Crime, Drama]",1984,3.0


Lets drop some columns that we do not need such as genres and year

In [0]:
#Dropping information we don't need such as year and genres
Lawrence_movie_ratings = Lawrence_movie_ratings.drop(['genres','year'], 1)


#Final input dataframe
#If a movie you added in above isn't here, then it might not be in the original 
#dataframe or it might spelled differently, please check capitalisation.
Lawrence_movie_ratings

,movieId,title,rating
0,4,Waiting to Exhale,3.9
1,1350,"Omen, The",5.0
2,45662,"Omen, The",5.0
3,1997,"Exorcist, The",4.8
4,2153,"Avengers, The",4.5
5,89745,"Avengers, The",4.5
6,3409,Final Destination,4.9
7,3527,Predator,4.9
8,4085,Beverly Hills Cop,3.0


Step 2: Learning Lawrence's Profile

We're going to start by learning the input's preferences, so let's get the subset of movies that the input has watched from the Dataframe containing genres defined with binary values.

In [0]:
# filter the selection by outputing movies that exist in both lawrence_movie_ratings and movies_with_genres
Lawrence_genres_df = movies_with_genres[movies_with_genres.movieId.isin(Lawrence_movie_ratings.movieId)]
Lawrence_genres_df

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
3,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1038,1350,"Omen, The","[Horror, Mystery, Thriller]",1976,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1472,1997,"Exorcist, The","[Horror, Mystery]",1973,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1611,2153,"Avengers, The","[Action, Adventure]",1998,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2547,3409,Final Destination,"[Drama, Thriller]",2000,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2636,3527,Predator,"[Action, Sci-Fi, Thriller]",1987,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3051,4085,Beverly Hills Cop,"[Action, Comedy, Crime, Drama]",1984,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6216,45662,"Omen, The","[Horror, Thriller]",2006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7693,89745,"Avengers, The","[Action, Adventure, Sci-Fi, IMAX]",2012,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


We'll only need the actual genre table, so let's clean this up a bit by resetting the index and dropping the movieId, title, genres and year columns.

In [0]:
# First, let's reset index to default and drop the existing index.
Lawrence_genres_df.reset_index(drop=True, inplace=True)

# Next, let's drop redundant columns
Lawrence_genres_df.drop(['movieId','title','genres','year'], axis=1, inplace=True)

# Let's view chamges

Lawrence_genres_df

/usr/local/lib/python3.6/dist-packages/pandas/core/frame.py:3940: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  errors=errors)


,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
0,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
6,0.0,0.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
8,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


Step 3: Building Lawrence's Profile<br>
To do this, we're going to turn each genre into weights, by multiplying Lawrence's movie ratings by lawrence_genres_df table. And then summing up the resulting table by column. This operation is actually a dot product between a matrix and a vector.
First let's confirm the shapes of the data frames we have recently defined

In [0]:
# let's confirm the shapes of our data frames to guide us as we do matrix multiplication

print('Shape of Lawrence_movie_ratings is:',Lawrence_movie_ratings.shape)
print('Shape of Lawrence_genres_df is:',Lawrence_genres_df.shape)

Shape of Lawrence_movie_ratings is: (9, 3)
Shape of Lawrence_genres_df is: (9, 20)


In [0]:
# Let's find the dot product of transpose of Lawrence_genres_df by lawrence rating column
Lawrence_profile = Lawrence_genres_df.T.dot(Lawrence_movie_ratings.rating)

# Let's see the result
Lawrence_profile

Adventure              7.8
Animation              0.0
Children               0.0
Comedy                 8.8
Fantasy                0.0
Romance                3.9
Drama                 13.3
Action                17.2
Crime                  4.9
Thriller              18.9
Horror                14.9
Mystery               10.0
Sci-Fi                 7.5
War                    0.0
Musical                0.0
Documentary            0.0
IMAX                   3.0
Western                0.0
Film-Noir              0.0
(no genres listed)     0.0
dtype: float64

Just by Eye-balling his profile, it is clear that Lawrence loves 'Thriller', 'Action' and 'Horror' movies the most… apt as can be.<br>
Now, we have the weights for all his preferences. This is known as the User Profile. We can now recommend movies that satisfy Lawrence.<br>
Let's start by editing the original movies_with_genres data frame that contains all movies and their genres columns.

Step 4: Deploying The Content-Based Recommender System.

In [0]:
# let's set the index to the movieId
movies_with_genres = movies_with_genres.set_index(movies_with_genres.movieId)

# let's view the head
movies_with_genres.head()

,movieId,title,genres,year,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,,,,,
1,1,Toy Story,"[Adventure, Animation, Children, Comedy, Fantasy]",1995,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,2,Jumanji,"[Adventure, Children, Fantasy]",1995,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,3,Grumpier Old Men,"[Comedy, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,4,Waiting to Exhale,"[Comedy, Drama, Romance]",1995,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,5,Father of the Bride Part II,[Comedy],1995,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


Let's delete irrelevant columns from the movies_with_genres data frame that contains all 9742 movies and distinctive columns of genres.

In [0]:
# Deleting four unnecessary columns.
movies_with_genres.drop(['movieId','title','genres','year'], axis=1, inplace=True)

# Viewing changes.
movies_with_genres.head()

,Adventure,Animation,Children,Comedy,Fantasy,Romance,Drama,Action,Crime,Thriller,Horror,Mystery,Sci-Fi,War,Musical,Documentary,IMAX,Western,Film-Noir,(no genres listed)
movieId,,,,,,,,,,,,,,,,,,,,
1,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


With Lawrence's profile and the complete list of movies and their genres in hand, we're going to take the weighted average of every movie based on his profile and recommend the top twenty movies that match his preference.

In [0]:
# Multiply the genres by the weights and then take the weighted average.
recommendation_table_df = (movies_with_genres.dot(Lawrence_profile)) / Lawrence_profile.sum()

# Let's view the recommendation table
recommendation_table_df.head()

movieId
1    0.150635
2    0.070780
3    0.115245
4    0.235935
5    0.079855
dtype: float64

Let's sort the recommendation table in descending order

In [0]:
# Let's sort values from great to small
recommendation_table_df.sort_values(ascending=False, inplace=True)

#Just a peek at the values
recommendation_table_df.head(20)

movieId
81132    0.869328
43932    0.742287
7235     0.707804
36509    0.692377
79132    0.678766
51545    0.663339
198      0.651543
6395     0.651543
54771    0.651543
74685    0.651543
26701    0.651543
4956     0.634301
60684    0.634301
4210     0.627949
71999    0.622505
31804    0.621597
2617     0.613430
72165    0.613430
91542    0.613430
27683    0.610708
dtype: float64

Now here's the recommendation table! Complete with movie details and genres for the top 20 movies that match Lawrence's profile.

In [0]:
# first we make a copy of the original movies_df
copy = movies_df.copy(deep=True)

# Then we set its index to movieId
copy = copy.set_index('movieId', drop=True)

# Next we enlist the top 20 recommended movieIds we defined above
top_20_index = recommendation_table_df.index[:20].tolist()

# finally we slice these indices from the copied movies df and save in a variable
recommended_movies = copy.loc[top_20_index, :]

# Now we can display the top 20 movies in descending order of preference
recommended_movies

,title,genres,year
movieId,,,
81132,Rubber,"[Action, Adventure, Comedy, Crime, Drama, Film...",2010
43932,Pulse,"[Action, Drama, Fantasy, Horror, Mystery, Sci-...",2006
7235,Ichi the Killer (Koroshiya 1),"[Action, Comedy, Crime, Drama, Horror, Thriller]",2001
36509,"Cave, The","[Action, Adventure, Horror, Mystery, Sci-Fi, T...",2005
79132,Inception,"[Action, Crime, Drama, Mystery, Sci-Fi, Thrill...",2010
51545,Pusher III: I'm the Angel of Death,"[Action, Comedy, Drama, Horror, Thriller]",2005
198,Strange Days,"[Action, Crime, Drama, Mystery, Sci-Fi, Thriller]",1995
6395,"Crazies, The (a.k.a. Code Name: Trixie)","[Action, Drama, Horror, Sci-Fi, Thriller]",1973
54771,"Invasion, The","[Action, Drama, Horror, Sci-Fi, Thriller]",2007


Run this cell below to kill the note book and free up space in colab

In [0]:
#import os, signal
#os.kill(os.getpid(), signal.SIGKILL)